In [1]:
from dig.threedgraph.dataset import QM93D
from dig.threedgraph.method import SphereNet, ComENet, DimeNetPP, ProNet, SchNet
from dig.threedgraph.evaluation import ThreeDEvaluator
from dig.threedgraph.method import run
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from tqdm import tqdm
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, Batch
from rdkit import Chem
from rdkit.Chem import AllChem
from torch_sparse import SparseTensor
import utils
import tts
# название модели, под которым оно будет сохраняться в логах
name_of_model = 'test'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [2]:
# отдельная функция для подчистки датасета, убирает с отрицательной токсичностью и заменяет в некоторых названия xyz
dataset = utils.clean_dataset(pd.read_csv("dataset/md_dataset_full_v3.csv"))

Shape of old dataset: (34926, 56)
Processing...


100%|██████████| 56/56 [00:02<00:00, 22.60it/s]

Shape of new dataset: (34915, 56)


In [3]:
suspects = pd.DataFrame()

for i, row in dataset.iterrows():
    if len(row['xyz_0'].split("\n")) == 18:
        suspects = pd.concat([suspects, row.to_frame().T], ignore_index=True)

In [3]:
def xyz_strin(xyz_string, target, mol_id, i):
    lines = xyz_string.strip().split('\n')
    coords = []
    atom_types = []
    for line in lines:
        atom_data = line.strip().split()
        atom_type = atom_data[0]
        x, y, z = map(float, atom_data[1:4])
        coords.append([x, y, z])
        atom_types.append(Chem.Atom(atom_type))
    pos = torch.tensor(coords)
    z = torch.tensor([atom.GetAtomicNum() for atom in atom_types])
    
    return Data(pos=pos, z=z, y=target, mol_id=mol_id, i=i)

X = []

for _, row in tqdm(dataset.iterrows(), total=len(dataset)):
    for i in range(51):
        xyz_string = row[f'xyz_{i}']
        try:
            data = xyz_strin(xyz_string, torch.tensor([row['mouse_intraperitoneal_LD50']]), row['mol_id'], i)
        except RuntimeError:
            print(row)
            continue
        X.append(data)

100%|██████████| 34923/34923 [08:16<00:00, 70.32it/s]


In [3]:
# я решил протестировать несколько вариаций датасетов, чтобы понять, есть ли какая-то разница
X_train, X_val, X_test, y_train, y_val, y_test = tts.create_tts_def(dataset, n=2)

100%|██████████| 34915/34915 [00:21<00:00, 1659.49it/s]


In [6]:
len(X_train), len(X_val), len(X_test)

(24446, 6985, 3493)

In [4]:
len(X_train), len(X_val), len(X_test)

(48880, 13966, 6984)

In [9]:
model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4,
                  hidden_channels=128, out_channels=1, int_emb_size=64,
                  basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
                  num_spherical=3, num_radial=6, envelope_exponent=5,
                  num_before_skip=1, num_after_skip=2, num_output_layers=3)
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

In [5]:
model = SchNet(energy_and_force=False, cutoff=15, num_layers=1, hidden_channels=128, out_channels=1, num_filters=128, num_gaussians=50)

In [5]:
from torch.optim import Adam
run3d = run()
optimizer = Adam(model.parameters(), lr=0.0005, weight_decay=0)

In [6]:
model.train()
model = model.to(device)

In [7]:
bads = []
for i in tqdm(range(len(X))):
    try:
        out = model(Batch.from_data_list([X[i]]).to(device))
    except RuntimeError as e:
        print(f"Error processing sample {X[i]}")
        bads.append(X[i])

  2%|▏         | 30145/1781073 [01:44<1:48:10, 269.76it/s]

Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=1)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=2)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=3)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=4)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=5)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=6)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=7)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=8)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=9)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=10)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=11)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=12)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=13)
Error pr

  2%|▏         | 30173/1781073 [01:44<2:55:24, 166.36it/s]

Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=18)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=19)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=20)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=21)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=22)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=23)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=24)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=25)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=26)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=27)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=28)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=29)
Error processing sample Data(y=[1], pos=[26, 3], z=[26], mol_id=19060, i=30)

  2%|▏         | 30195/1781073 [01:44<3:26:05, 141.59it/s]

...

 99%|█████████▉| 1761344/1781073 [1:39:28<01:15, 261.49it/s]  

Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=1)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=2)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=3)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=4)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=5)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=6)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=7)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=8)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=9)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=10)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=11)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=12)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=13)
Error pr

 99%|█████████▉| 1761372/1781073 [1:39:28<02:00, 164.04it/s]

Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=34)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=35)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=36)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=37)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=38)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=39)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=40)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=41)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=42)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=43)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=44)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=45)
Error processing sample Data(y=[1], pos=[32, 3], z=[32], mol_id=44018, i=46)

100%|██████████| 1781073/1781073 [1:40:33<00:00, 295.22it/s]


In [10]:
len(bads)/50

8.0

In [22]:
mols_ids = set([float(x.y[0]) for x in bads])
mols_ids

{2.54211688041687,
 2.6342971324920654,
 2.9159841537475586,
 3.0732991695404053,
 3.227167844772339,
 4.15582275390625,
 4.653680324554443}

In [7]:
# Обучение модели

run3d.run(device, X_train, X_val, X_test, model, loss_func, evaluation,
          epochs=80, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15,
          save_dir=f"models/{name_of_model}", log_dir=f"logs/{name_of_model}")

#Params: 1890118

=====Epoch 1

Training...


100%|██████████| 764/764 [01:38<00:00,  7.77it/s]



Evaluating...



100%|██████████| 219/219 [00:14<00:00, 14.89it/s]



Testing...


...



Evaluating...



100%|██████████| 219/219 [00:14<00:00, 15.20it/s]



Testing...



100%|██████████| 110/110 [00:07<00:00, 14.86it/s]


{'Train': 0.03171477442197697, 'Validation': 0.3804886043071747, 'Test': 0.37855228781700134}
Best validation MAE so far: 0.37842077016830444
Test MAE when got best validation result: 0.3755556046962738


In [25]:
# тетстовый код, чтобы можно было подгрузить модель после ее сохранения
path = 'models/valid_checkpoint.pt'
model1 = SphereNet(energy_and_force=True, cutoff=5.0, num_layers=4,
                  hidden_channels=128, out_channels=1, int_emb_size=64,
                  basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,
                  num_spherical=3, num_radial=6, envelope_exponent=5,
                  num_before_skip=1, num_after_skip=2, num_output_layers=3)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model1.to(device)
checkpoint = torch.load(path, map_location=device)
model1.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [12]:
X_train[0]

Data(y=[1], pos=[33, 3], z=[33])

In [26]:
# тестовый код, как можно делать подсчет метрики с помощью библиотеки
valid_loader = DataLoader(X_test, 32, shuffle=False)
run3d.val(model, valid_loader, False, 100, evaluation, device)

100%|██████████| 1/1 [00:00<00:00, 20.98it/s]


23.90863037109375

In [ ]:
dataset.mouse_intraperitoneal_LD50.describe()

Дальше я тестировал, получится ли при такой же конвертации закинуть в другие модели для сравнения результатов. Во все модели бибилиотеки, кроме ProNet закинуть обработанный датасет можно.

In [10]:
model2 = SchNet(energy_and_force=False, cutoff=10.0, num_layers=6, hidden_channels=128, out_channels=1, num_filters=128, num_gaussians=50)
# model3 = ProNet(level='aminoacid', num_blocks=4, hidden_channels=128, out_channels=1, mid_emb=64, num_radial=6, num_spherical=2, cutoff=10.0, max_num_neighbors=32, int_emb_layers=3, out_layers=2, num_pos_emb=16, dropout=0, data_augment_eachlayer=False, euler_noise=False)
model4 = DimeNetPP(energy_and_force=False, cutoff=5.0, num_layers=4, hidden_channels=128, out_channels=1, int_emb_size=64, basis_emb_size=8, out_emb_channels=256, num_spherical=7, num_radial=6, envelope_exponent=5, num_before_skip=1, num_after_skip=2, num_output_layers=3, output_init='GlorotOrthogonal')
model5 = ComENet(cutoff=8.0, num_layers=4, hidden_channels=256, middle_channels=64, out_channels=1, num_radial=3, num_spherical=2, num_output_layers=3)

In [16]:
run3d = run()
run3d.run(device, X_train, X_val, X_test, model5, loss_func, evaluation,
          epochs=10, batch_size=32, vt_batch_size=32, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15,
          save_dir=f"models/{name_of_model}", log_dir=f"logs/{name_of_model}")

#Params: 3778817

=====Epoch 1

Training...


  0%|          | 111/38199 [00:06<39:57, 15.89it/s] 


KeyboardInterrupt: 